# Defect Classifier


In [1]:
import os
import numpy as np
from pathlib import Path
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers

for gpu in tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)


In [2]:
# Config
DATASET_PATH = Path('/Users/willekjellberg/.cache/kagglehub/datasets/zhangyunsheng/defects-class-and-location/versions/1/images/images')
CLASSES = ['crease', 'crescent_gap', 'inclusion', 'oil_spot', 'punching_hole', 'rolled_pit', 'silk_spot']
IMG_SIZE = 128

def load_image(path):
    img = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None
    # Crop black borders
    mask = img > 10
    if mask.any():
        rows, cols = np.where(mask)
        img = img[rows.min():rows.max()+1, cols.min():cols.max()+1]
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    # CLAHE + z-score + min-max to [0,1]
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    img = clahe.apply(img)
    img = img.astype(np.float32)
    mean, std = np.mean(img), np.std(img)
    if std > 0:
        img = (img - mean) / std
    mn, mx = img.min(), img.max()
    if mx > mn:
        img = (img - mn) / (mx - mn)
    return img[..., np.newaxis]


In [3]:
# Load data
X, y = [], []
for i, cls in enumerate(CLASSES):
    for path in (DATASET_PATH / cls).glob('*.jpg'):
        img = load_image(path)
        if img is not None:
            X.append(img)
            y.append(i)

X, y = np.array(X), np.array(y)
print(f"Loaded {len(X)} images, {len(CLASSES)} classes")

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Class weights for imbalanced data
class_counts = np.bincount(y_train)
class_weights = {i: len(y_train) / (len(CLASSES) * c) for i, c in enumerate(class_counts)}


Loaded 1598 images, 7 classes


In [4]:
# Model
model = tf.keras.Sequential([
    layers.Input((IMG_SIZE, IMG_SIZE, 1)),
    layers.RandomRotation(0.2),
    layers.RandomTranslation(0.15, 0.15),
    layers.RandomZoom(0.15),
    
    layers.Conv2D(32, 3, activation='relu', padding='same'),
    layers.Conv2D(32, 3, activation='relu', padding='same'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    
    layers.Conv2D(64, 3, activation='relu', padding='same'),
    layers.Conv2D(64, 3, activation='relu', padding='same'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    
    layers.Conv2D(128, 3, activation='relu', padding='same'),
    layers.Conv2D(128, 3, activation='relu', padding='same'),
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(CLASSES), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


2026-01-07 12:17:18.140812: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2026-01-07 12:17:18.140838: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2026-01-07 12:17:18.140843: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 25.92 GB
2026-01-07 12:17:18.140861: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-01-07 12:17:18.140874: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
# Train
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(2048).batch(32).prefetch(2)
val_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32).prefetch(2)

history = model.fit(
    train_ds, 
    validation_data=val_ds,
    epochs=int(os.getenv("EPOCHS", "30")),
    class_weight=class_weights,
    callbacks=[
        tf.keras.callbacks.EarlyStopping('val_accuracy', patience=15, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau('val_accuracy', factor=0.5, patience=5)
    ]
)


2026-01-07 12:17:18.994576: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


 1/40 ━━━━━━━━━━━━━━━━━━━━ 1:46 3s/step - accuracy: 0.0938 - loss: 2.0287

 2/40 ━━━━━━━━━━━━━━━━━━━━ 3s 80ms/step - accuracy: 0.1250 - loss: 1.9764

 3/40 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - accuracy: 0.1458 - loss: 1.9527

 4/40 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - accuracy: 0.1562 - loss: 1.9937

...

37/40 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.1445 - loss: 1.9942

38/40 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.1444 - loss: 1.9932

39/40 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.1444 - loss: 1.9921

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.1445 - loss: 1.9910

40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 79ms/step - accuracy: 0.1446 - loss: 1.9900 - val_accuracy: 0.4062 - val_loss: 1.9404 - learning_rate: 0.0010


In [6]:
# Evaluate
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {acc:.1%}")


Test Accuracy: 40.6%


In [7]:
# Save
model.save('defect_classifier_model.keras')
